In [20]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os


In [21]:
def detect_and_predict_mask(frame,faceNet,maskNet):
    # grab dimensions
    (h,w) = frame.shape[:2]
    # blob construction
    blob = cv2.dnn.blobFromImage(frame,1.0,(244,244),(104.0,177.0,123.0))
    
    # pass the blob thru network and obtain the face detections
    faceNet.setInput(blob)
    detections = faceNet.forward()
    print(detections.shape)
    
    faces =[]
    preds =[]
    locs =[]
    
    # loop over the detections
    for i in range(0,detections.shape[2]):
        confidence = detections[0,0,i,2]
        
        # filter weak detections
        if confidence > 0.5:
                box = detections[0,0,1,3:7] * np.array([w,h,w,h])
                (startx,starty,endx,endy) = box.astype('int')
                
                (startx,starty ) = (max(0,startx), max(0,starty))
                (endx,endy) = (min(w-1,endx),min(h-1,endy))
                
                
                
                
                face = frame[startx:endx ,starty:endy ]
                face = cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
                face = cv2.resize(face,(244,244))
                face = img_to_array(face)
                face = preprocess_input(face)
                
                
                
                face.append(face)
                locs.append((startx,starty,endx,endy))
    # only predict if one face is seen
    if len(faces)> 0:
        faces = np.array(faces,dtype = 'float32')
        preds = maskNet.predict(faces ,batch_size =32)
        
    return (locs,preds)
                
    
    

In [22]:
# load our face detection model from disk
facepath  = 'face_detector/deploy.prototxt'
weightpath = 'face_detector/res10_300x300_ssd_iter_140000.caffemodel'
faceNet = cv2.dnn.readNet(facepath,weightpath)

In [23]:
# load face mask model from disk
maskNet = load_model('mask_detector.model')


In [24]:
# initialize the video stream

print('[info] starting video stream...')
vs = VideoStream(src=0).start()


# loop over the frames of the video stream
while True:
    # grab frame from video stream and resize it to max width of 400 pixels
    frame = vs.read()
    frame = imutils.resize(frame , width = 400)

    # detect face in frame and determine if they are wearing a mask or not
    (locs,pred) = detect_and_predict_mask(frame,faceNet,maskNet)

    # loop over the detected face locations

    for (box,pred ) in zip(locs,pred):
        # unpack bounding box and predictions
        (startx,starty,endx,endy) = box
        (mask,withoutmask) = pred

        # determine the box label and color 
        label = 'mask' if mask>withoutmask else 'withoutmask'
        color =(0,255,0) if mask> withoutmask else (0,0,255)

        # include the probability in the label
        label ='{} : {:.2f}%'.format(label,max(mask,withoutmask)* 100)


        # display the label and the bounding box on the resultant frame
        cv2.putText(frame,label,(startx,starty-10), cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(frame,(startx,starty),(endx,endy),color,2)

    # show the output frqme
    cv2.imshow('FRAME',frame)
    key = cv2.waitKey(1) & 0xFF


    # if the 's' key was pressed ,break from loop
    if key == ord('s'):
        break

# cleanup
cv2.destroyAllWindows()
vs.stop()




[info] starting video stream...
(1, 1, 200, 7)


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [25]:
key = 's'